<a href="https://colab.research.google.com/github/hij826/deep_learning/blob/master/CifarToMnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init

In [0]:
#cuda 사용할 때 쓰고 아니면 cpu 사용
device = 'cuda' if torch.cuda.is_available() else 'cpu'


In [0]:

trainset = dsets.MNIST(root='./data',
                       train=True,
                       download=True,
                       transform=transforms.ToTensor())
trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=50,
                                          shuffle=True)

testset = dsets.MNIST(root='./data',
                      train=False,
                      download=True,
                      transform=transforms.ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=50,
                                         shuffle=False)

classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

In [0]:
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #첫 번째 convolution layer
        #1 channel 받아서 32개의 channel 내는데 kernel size는 3, padding은 1
        self.conv1 = nn.Conv2d(1, 32, 3, padding = 1)  #28x28x32
        #pooling층
        #max pooling으로 2x2
        self.pool = nn.MaxPool2d(2, 2) #14x14x32
        #두 번째 convolution layer
        #32개의 channel 받아서 64개의 channel 내는데 kernel size 3, paading 1
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1) #14x14x64 , max pooling 후 7x7x64
        
        #첫 번째 fully connected layer
        self.fc1 = nn.Linear(7*7*64, 120)
        #두 번째 fully connected layer
        self.fc2 = nn.Linear(120, 10) # minist 총 출력이 0~9기 때문에 10
        #self.fc3 = nn.Linear(84, 10)
        #torch.nn.init.xavier_uniform_(self.fc1.weight)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) #첫번째 layer통과
        x = self.pool(F.relu(self.conv2(x))) #두번째 layer통과
        x = x.view(-1, 7*7*64) # 한 줄로 펼쳐서 batch_size x 3136 되어야 함.
        x = self.fc1(x)
        #x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x)) # fully connected layer 다 통과 후 결과
        #x = self.fc3(x)
        return x

#모델 생성. gpu 사용 위해서 to(device) 꼭 추가
net = Net().to(device)

In [0]:
#loss function optimizer 선언
import torch.optim as optim

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.005, momentum=0.9) #SGD optimizer 사용함

In [0]:
for epoch in range(3):  # 데이터셋을 수차례 반복합니다.

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # 입력을 받은 후
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # 변화도(Gradient) 매개변수를 0으로 만든 후
        optimizer.zero_grad()

        # 순전파 + 역전파 + 최적화
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # 통계 출력
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

[1,   200] loss: 0.045
[1,   400] loss: 0.043
[1,   600] loss: 0.041
[1,   800] loss: 0.043
[1,  1000] loss: 0.038
[1,  1200] loss: 0.045
[2,   200] loss: 0.035
[2,   400] loss: 0.040
[2,   600] loss: 0.036
[2,   800] loss: 0.037
[2,  1000] loss: 0.034
[2,  1200] loss: 0.041
[3,   200] loss: 0.035
[3,   400] loss: 0.039
[3,   600] loss: 0.031
[3,   800] loss: 0.036
[3,  1000] loss: 0.031
[3,  1200] loss: 0.033
Finished Training


In [0]:
#전체 데이터셋 정확도
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        labels = labels.to(device)
        outputs = net(images.to(device))
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 98 %


In [0]:

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        labels = labels.to(device)
        outputs = net(images.to(device))
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of     0 : 98 %
Accuracy of     1 : 99 %
Accuracy of     2 : 98 %
Accuracy of     3 : 97 %
Accuracy of     4 : 98 %
Accuracy of     5 : 100 %
Accuracy of     6 : 97 %
Accuracy of     7 : 97 %
Accuracy of     8 : 100 %
Accuracy of     9 : 97 %
